In [1]:
import argparse

In [2]:
%load_ext autoreload
%autoreload 2

from load_data import load_data
from model import Optimizer, nihgcn
from myutils import *
from sampler import RandomSampler
from sklearn.model_selection import KFold

In [3]:
class Args:
    def __init__(self):
        self.device = "cpu"  # cuda:number or cpu
        self.data = "nci"  # Dataset{gdsc or ccle}
        self.lr = 0.001  # the learning rate
        self.wd = 1e-5  # the weight decay for l2 normalizaton
        self.layer_size = [1024, 1024]  # Output sizes of every layer
        self.alpha = 0.25  # the scale for balance gcn and ni
        self.gamma = 8  # the scale for sigmod
        self.epochs = 1000  # the epochs for model


args = Args()

In [4]:
res, drug_finger, exprs, null_mask, pos_num = load_data(args)

load nci


In [5]:
k = 5
kfold = KFold(n_splits=k, shuffle=True, random_state=42)

for train_index, test_index in kfold.split(np.arange(pos_num)):
    sampler = RandomSampler(res, train_index, test_index, null_mask)
    model = nihgcn(
        adj_mat=sampler.train_data,
        cell_exprs=exprs,
        drug_finger=drug_finger,
        layer_size=args.layer_size,
        alpha=args.alpha,
        gamma=args.gamma,
        device=args.device,
    ).to(args.device)
    opt = Optimizer(
        model,
        sampler.train_data,
        sampler.test_data,
        sampler.test_mask,
        sampler.train_mask,
        roc_auc,
        lr=args.lr,
        wd=args.wd,
        epochs=args.epochs,
        device=args.device,
    ).to(args.device)
    true_data, predict_data = opt()
    true_datas = pd.concat([true_datas, pd.DataFrame(true_data)], ignore_index=True)
    predict_datas = pd.concat(
        [predict_datas, pd.DataFrame(predict_data)], ignore_index=True
    )

epoch:   0 loss:0.703196 auc:0.4800
epoch:  20 loss:0.339121 auc:0.8554
epoch:  40 loss:0.283475 auc:0.8638
epoch:  60 loss:0.256080 auc:0.8620
epoch:  80 loss:0.240256 auc:0.8594
epoch: 100 loss:0.231027 auc:0.8569
epoch: 120 loss:0.225951 auc:0.8548
epoch: 140 loss:0.223455 auc:0.8533
epoch: 160 loss:0.221809 auc:0.8512
epoch: 180 loss:0.221129 auc:0.8507
epoch: 200 loss:0.220258 auc:0.8502
epoch: 220 loss:0.219927 auc:0.8507
epoch: 240 loss:0.219314 auc:0.8500
epoch: 260 loss:0.219367 auc:0.8494
epoch: 280 loss:0.219411 auc:0.8491
epoch: 300 loss:0.218894 auc:0.8488
epoch: 320 loss:0.219258 auc:0.8484
epoch: 340 loss:0.218228 auc:0.8489


KeyboardInterrupt: 